In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import time

In [2]:
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
time.sleep(2)

In [5]:
html = browser.html

In [6]:
soup = BeautifulSoup(html, 'html.parser')

In [7]:
slideElement = soup.select_one('ul.item_list li.slide')
slideElement



<li class="slide"><div class="image_and_description_container"><a href="/news/8450/the-mast-is-raised-for-nasas-mars-2020-rover/" target="_self"><div class="rollover_description"><div class="rollover_description_inner">Engineers at JPL take a group selfie after attaching the remote sensing mast to the Mars 2020 rover.</div><div class="overlay_arrow"><img alt="More" src="/assets/overlay-arrow.png"/></div></div><div class="list_image"><img alt="Engineers and technicians at NASA's Jet Propulsion Laboratory in Pasadena, California, install the remote sensing mast on the Mars 2020 rover." src="/system/news_items/list_view_images/8450_PIA23268-320x240.jpg"/></div><div class="bottom_gradient"><div><h3>The Mast Is Raised for NASA's Mars 2020 Rover</h3></div></div></a><div class="list_text"><div class="list_date">June 14, 2019</div><div class="content_title"><a href="/news/8450/the-mast-is-raised-for-nasas-mars-2020-rover/" target="_self">The Mast Is Raised for NASA's Mars 2020 Rover</a></div><

In [8]:
New_Title= slideElement.find("div", class_='content_title').get_text()

In [9]:
New_Paragraph = slideElement.find("div", class_='article_teaser_body').get_text()

New_Paragraph

'Engineers at JPL take a group selfie after attaching the remote sensing mast to the Mars 2020 rover.'

In [10]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
time.sleep(2)

In [11]:
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()


In [12]:
import time
time.sleep(3)

In [13]:
more_info_elem = browser.find_link_by_partial_text('more info')
more_info_elem.click()
time.sleep(2)
html2 = browser.html
img_soup = BeautifulSoup(html2, 'html.parser')


In [14]:
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel 




'/spaceimages/images/largesize/PIA16105_hires.jpg'

In [15]:
category = img_soup.select_one('figure.lede')

Partial_url = category.find('a')['href']


In [16]:
Fullimage_url = 'https://www.jpl.nasa.gov' + Partial_url

Fullimage_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16105_hires.jpg'

In [17]:
def scrape_hemisphere(html_text):

    # Soupify the html text
    hemi_soup = BeautifulSoup(html_text, "html.parser")

    # Try to get href and text except if error.
    try:
        title_elem = hemi_soup.find("h2", class_="title").get_text()
        sample_elem = hemi_soup.find("a", text="Sample").get("href")

    except AttributeError:

        # Image error returns None for better front-end handling
        title_elem = None
        sample_elem = None

    hemisphere = {
        "title": title_elem,
        "img_url": sample_elem
    }

    return hemisphere

In [18]:
# A way to break up long strings
    url = (
        "https://astrogeology.usgs.gov/search/"
        "results?q=hemisphere+enhanced&k1=target&v1=Mars"
    )

    browser.visit(url)
    time.sleep(2)

    # Click the link, find the sample anchor, return the href
    hemisphere_image_urls = []
    for i in range(4):

        # Find the elements on each loop to avoid a stale element exception
        browser.find_by_css("a.product-item h3")[i].click()
        time.sleep(2)

        hemi_data = scrape_hemisphere(browser.html)

        # Append hemisphere object to list
        hemisphere_image_urls.append(hemi_data)

        # Finally, we navigate backwards
        browser.back()

In [19]:
print (hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [20]:
import pandas as pd
df = pd.read_html('http://space-facts.com/mars/')[0]
df.columns=['description', 'value']
df.set_index('description', inplace=True)
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [21]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>

In [22]:
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

html = browser.html


In [23]:
weather_soup = BeautifulSoup(html, 'html.parser')
# First, find a tweet with the data-name `Mars Weather`
mars_weather_tweet = weather_soup.find('div', attrs={"class": "tweet", "data-name": "Mars Weather"})
mars_weather = mars_weather_tweet.find('p', 'tweet-text').get_text()
mars_weather

'InSight sol 195 (2019-06-14) low -105.6ºC (-158.1ºF) high -23.8ºC (-10.8ºF)\nwinds from the SSE at 4.3 m/s (9.7 mph) gusting to 14.5 m/s (32.5 mph)\npressure at 7.60 hPapic.twitter.com/3LNNz5uXCd'